https://github.com/pranavphoenix/BiLSTM-POS-Tagging/blob/main/BiLSTM_POS_Tagging.ipynb

TODO:

- evitare di utilizzare la punctuation nel calcolo delle metriche, magari utilizzando l'array di pesi 'sample_weight' che si trova nell'altro notebook;

- Provare se i risultati migliorano con preprocessing (e.g. lowerando le parole)+ preprocessing di termini strani;
- Aggiustare il notebook perché fa cagare;

In [2]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
from collections import defaultdict
from keras import backend as K
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed
from keras.layers import Embedding, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences


import urllib.request
import zipfile
import progressbar

In [3]:
#Downloading the dataset
nltk.download('treebank')

data = nltk.corpus.treebank.tagged_sents()

#Downloading Glove Word Embeddings
pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

# Download the GloVe embeddings file
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
urllib.request.urlretrieve(url, 'glove.6B.zip', show_progress)

# Extract the zip file
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
100% (862182613 of 862182613) |##########| Elapsed Time: 0:02:44 Time:  0:02:44


In [4]:
#Function to ignore the 0 padding while calculating accuracy
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

#Function to return one code encoding of tags
def one_hot_encoding(tag_sents, n_tags):
    tag_one_hot_sent = []
    for tag_sent in tag_sents:
        tags_one_hot = []
        for tag in tag_sent:
            tags_one_hot.append(np.zeros(n_tags))
            tags_one_hot[-1][tag] = 1.0
        tag_one_hot_sent.append(tags_one_hot)
    return np.array(tag_one_hot_sent)

#Function to convert output into tags
def logits_to_tags(tag_sentences, index):
    tag_sequences = []
    for tag_sentence in tag_sentences:
        tag_sequence = []
        for tag in tag_sentence:
            # if index[np.argmax(tag)] == "-PAD-":
            #     break
            # else:
                tag_sequence.append(index[np.argmax(tag)])
        tag_sequences.append(np.array(tag_sequence))
    return tag_sequences

In [5]:
# Get the files' list
fileids = nltk.corpus.treebank.fileids()

# Get the Penn Treebank corpus and tokenize the text
train_corpus = nltk.corpus.treebank.tagged_sents(fileids[:100])
val_corpus = nltk.corpus.treebank.tagged_sents(fileids[100:150])
test_corpus = nltk.corpus.treebank.tagged_sents(fileids[150:])

# Flatten the lists
ignore = [':', '"', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM']

# train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist if item[1] != '-NONE-']
# val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist if item[1] != '-NONE-']
# test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist if item[1] != '-NONE-']

train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist]
val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist]
test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist]

train_df = pd.DataFrame(train_corpus, columns = ['word', 'tag', 'sentence'])
val_df = pd.DataFrame(val_corpus, columns = ['word', 'tag', 'sentence'])
test_df = pd.DataFrame(test_corpus, columns = ['word', 'tag', 'sentence'])
print(train_df)

         word     tag sentence
0      Pierre     NNP        0
1      Vinken     NNP        0
2           ,       ,        0
3          61      CD        0
4       years     NNS        0
...       ...     ...      ...
50743      he     PRP     1962
50744     has     VBZ     1962
50745   faced     VBN     1962
50746   *T*-2  -NONE-     1962
50747       .       .     1962

[50748 rows x 3 columns]


In [6]:
tags_train = sorted(list(set([x for x in train_df.tag])))
tags_val = sorted(list(set([x for x in val_df.tag])))
tags_test = sorted(list(set([x for x in test_df.tag])))

print('Train tags number:',len(tags_train))
print('Val tags number:',len(tags_val))
print('Test tags number:',len(tags_test))


Train tags number: 46
Val tags number: 45
Test tags number: 41


###Vocabulary

GloVe Vocabulary (V1)

In [7]:
#Use the 300 dimensional GLove Word Embeddings
glove_dir = './'

embeddings_index =  {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


V1 + Training set OOV (V2)

In [8]:
embedding_dim = 300

def update_vocab(df,embeddings_index,embedding_dim):
  oov_c = 0
  for word in df.word:
    if word not in embeddings_index:
      oov_c += 1
      random_embed = np.random.rand(embedding_dim)
      embeddings_index[word] = random_embed
  print("Added",oov_c,"OOV words + respective embeddings to the vocabulary.")
  return embeddings_index

embeddings_index = update_vocab(train_df,embeddings_index,embedding_dim)

Added 2777 OOV words + respective embeddings to the vocabulary.


V2 + Validation set OOV (V3)

In [9]:
embeddings_index = update_vocab(val_df,embeddings_index,embedding_dim)

Added 951 OOV words + respective embeddings to the vocabulary.


V3 + Test set OOV (V4)

In [10]:
embeddings_index = update_vocab(test_df,embeddings_index,embedding_dim)

Added 455 OOV words + respective embeddings to the vocabulary.


In [11]:
#Building the actual word vocabulary

from collections import OrderedDict

idx2word = OrderedDict()
word2idx = OrderedDict()
    
curr_idx = 0
for key in embeddings_index.keys():
  word2idx[key] = curr_idx
  idx2word[curr_idx] = key
  curr_idx += 1

#word_listing = list(idx2word.values())

print(f'[Debug] Index -> Word vocabulary size: {len(idx2word)}')
print(f'[Debug] Word -> Index vocabulary size: {len(word2idx)}')

[Debug] Index -> Word vocabulary size: 404183
[Debug] Word -> Index vocabulary size: 404183


In [12]:
#Tag vocabulary

tag2idx = OrderedDict()

curr_id = 0

for tag in tags_train:
  tag2idx[tag] = curr_id
  curr_id += 1

print(f'[Debug] Tag -> Index vocabulary size: {len(tag2idx)}')

[Debug] Tag -> Index vocabulary size: 46


In [13]:
acc = []
conf_matrix = []

tag_list = tags_train
# The integers for each tag are the same as above

MAX_LENGTH = max([len(seq) for seq in data]) # maximum words in a sentence

conf_mat_df = pd.DataFrame(columns=tag_list, index=tag_list)
conf_mat_df = conf_mat_df.fillna(0)

In [14]:
train_sentences = train_df.groupby('sentence').word.apply(list).reset_index()['word']
val_sentences = val_df.groupby('sentence').word.apply(list).reset_index()['word']
test_sentences = test_df.groupby('sentence').word.apply(list).reset_index()['word']

train_tags = train_df.groupby('sentence').tag.apply(list).reset_index()['tag']
val_tags = val_df.groupby('sentence').tag.apply(list).reset_index()['tag']
test_tags = test_df.groupby('sentence').tag.apply(list).reset_index()['tag']

true_pos_tag = defaultdict(int)
false_pos_tag = defaultdict(int)
false_neg_tag = defaultdict(int)
precision_tags = defaultdict(float)
accuracy_tags = defaultdict(float)
recall_tags = defaultdict(float)
f1score_tags = defaultdict(float)


In [15]:
#Tokenising words and  by their indexes in vocabulary
train_sentences_X, test_sentences_X, val_sentences_X, train_tags_y, test_tags_y, val_tags_y = [], [], [], [], [], []

for sentence in train_sentences:
    sent_int = []
    for word in sentence:
        sent_int.append(word2idx[word])
   
    train_sentences_X.append(sent_int)

for sentence in val_sentences:
    sent_int = []
    for word in sentence:
        sent_int.append(word2idx[word])

    val_sentences_X.append(sent_int)

for sentence in test_sentences:
    sent_int = []
    for word in sentence:
        sent_int.append(word2idx[word])

    test_sentences_X.append(sent_int)

for sent_tags in train_tags:
    train_tags_y.append([tag2idx[tag] for tag in sent_tags])

for sent_tags in val_tags:
    val_tags_y.append([tag2idx[tag] for tag in sent_tags])

for sent_tags in test_tags:
    test_tags_y.append([tag2idx[tag] for tag in sent_tags])

#Add padding to sentences
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
val_sentences_X = pad_sequences(val_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')

train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
val_tags_y = pad_sequences(val_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')



In [16]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [17]:
print(tag2idx)

OrderedDict([('#', 0), ('$', 1), ("''", 2), (',', 3), ('-LRB-', 4), ('-NONE-', 5), ('-RRB-', 6), ('.', 7), (':', 8), ('CC', 9), ('CD', 10), ('DT', 11), ('EX', 12), ('FW', 13), ('IN', 14), ('JJ', 15), ('JJR', 16), ('JJS', 17), ('LS', 18), ('MD', 19), ('NN', 20), ('NNP', 21), ('NNPS', 22), ('NNS', 23), ('PDT', 24), ('POS', 25), ('PRP', 26), ('PRP$', 27), ('RB', 28), ('RBR', 29), ('RBS', 30), ('RP', 31), ('SYM', 32), ('TO', 33), ('UH', 34), ('VB', 35), ('VBD', 36), ('VBG', 37), ('VBN', 38), ('VBP', 39), ('VBZ', 40), ('WDT', 41), ('WP', 42), ('WP$', 43), ('WRB', 44), ('``', 45)])


In [20]:
weights = np.array([1,1,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0])
loss = weighted_categorical_crossentropy(weights)
print(weights.shape)

(46,)


In [21]:
#Building the Embedding Layer 
embedding_dim = 300

embedding_matrix = np.zeros((len(word2idx), embedding_dim))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if i < len(word2idx):
        embedding_matrix[i] = embedding_vector

#Building the BiLSTM model
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, ))) 
model.add(Embedding(len(word2idx), 300, weights=[embedding_matrix],trainable=False))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2idx))))
model.add(Activation('softmax'))

model.compile(loss=loss, optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
model.summary()
one_hot_train_tags_y = one_hot_encoding(train_tags_y, len(tag2idx))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 271, 300)          121254900 
                                                                 
 bidirectional (Bidirectiona  (None, 271, 512)         1140736   
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 271, 46)          23598     
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 271, 46)           0         
                                                                 
Total params: 122,419,234
Trainable params: 1,164,334
Non-trainable params: 121,254,900
_________________________________________________________________


In [22]:
history = model.fit(train_sentences_X, one_hot_encoding(train_tags_y, len(tag2idx)),\
          validation_data=(val_sentences_X, one_hot_encoding(val_tags_y, len(tag2idx))),
          batch_size=128, epochs= 15)


Epoch 1/15
16/16 [==============================] - 103s 6s/step - loss: 0.6496 - accuracy: 0.8501 - ignore_accuracy: 0.3408 - val_loss: 0.2659 - val_accuracy: 0.9193 - val_ignore_accuracy: 0.1736
Epoch 2/15
16/16 [==============================] - 92s 6s/step - loss: 0.2429 - accuracy: 0.9244 - ignore_accuracy: 0.2204 - val_loss: 0.2240 - val_accuracy: 0.9301 - val_ignore_accuracy: 0.2722
Epoch 3/15
16/16 [==============================] - 93s 6s/step - loss: 0.2102 - accuracy: 0.9339 - ignore_accuracy: 0.3302 - val_loss: 0.2027 - val_accuracy: 0.9353 - val_ignore_accuracy: 0.3326
Epoch 4/15
16/16 [==============================] - 89s 6s/step - loss: 0.1897 - accuracy: 0.9403 - ignore_accuracy: 0.3895 - val_loss: 0.1828 - val_accuracy: 0.9439 - val_ignore_accuracy: 0.4154
Epoch 5/15
16/16 [==============================] - 91s 6s/step - loss: 0.1694 - accuracy: 0.9459 - ignore_accuracy: 0.4490 - val_loss: 0.1618 - val_accuracy: 0.9487 - val_ignore_accuracy: 0.4678
Epoch 6/15
16/16 [=

In [23]:
scores = model.evaluate(test_sentences_X, one_hot_encoding(test_tags_y, len(tag2idx)))
acc.append(scores[2]*100)


predictions = model.predict(test_sentences_X)
pred_sequence = logits_to_tags(predictions, {i: t for t, i in tag2idx.items()})
#y_prob_class = model.predict_classes(test_sentences_X, verbose = 1)

21/21 [==============================] - 14s 638ms/step


In [35]:
ignore = [':', '-LRB-', '-RRB-', ',', '.', "''", '``']
for sen_num in range(len(test_tags)):
    for i,tag in enumerate(test_tags[sen_num]):
      
        conf_mat_df[tag][pred_sequence[sen_num][i]] +=1
        if test_tags[sen_num][i] == pred_sequence[sen_num][i]:
          true_pos_tag[tag] += 1
        else:
          false_neg_tag[tag] += 1
          false_pos_tag[pred_sequence[sen_num][i]] += 1

for tag in tag_list[1:]:
  if tag not in ignore:
    if (true_pos_tag[tag] + false_pos_tag[tag]) != 0:
      print(tag)
      precision_tags[tag] = true_pos_tag[tag] / (true_pos_tag[tag] + false_pos_tag[tag])
      recall_tags[tag] = true_pos_tag[tag] / (true_pos_tag[tag] + false_neg_tag[tag])
      f1score_tags[tag] = 2 * precision_tags[tag] * recall_tags[tag] / (precision_tags[tag] + recall_tags[tag])
      accuracy_tags[tag] = true_pos_tag[tag] / (true_pos_tag[tag] + false_neg_tag[tag] + false_pos_tag[tag])

$
-NONE-
CC
CD
DT
IN
JJ
JJR


ZeroDivisionError: ignored

In [27]:
print(len(f1score_tags))

7


In [26]:
sum = 0
for el in f1score_tags.items():
  sum += el[1]
print(sum/len(f1score_tags.items()))

0.8031544390336744
